In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from xclim.core.missing import missing_pct
from xclim.indices.generic import select_resample_op
from xclim.indices.stats import fa, fit, frequency_analysis, parametric_quantile
import gdown

In [ ]:
#'GOOGLE_DRIVE_LINK' to NetCDF file
url = 'https://drive.google.com/file/d/1ljDfwAeyfLNMsArQDOFP9-7y4EC7Brxq/view?usp=drive_link'
# Download the NetCDF file
file_path = 'data/input/ERA5_Tenerife_total_precipitation_day_1950_2000.nc'
gdown.download(url, file_path, quiet=False, fuzzy=True)

In [ ]:
# Load the ERA5 reanalysis daily precipitation data for Tenerife from 1950 - 2000 
file_path = 'data/input/ERA5_Tenerife_total_precipitation_day_1950_2000.nc'

daily_precipitation = xr.open_dataset(file_path)
# Add an attribute to the dataset
daily_precipitation.attrs['units'] = 'mm/d'
# create a array which only conains the years 
years = daily_precipitation['time'].dt.year.values

In [ ]:
# Print the loaded data base
daily_precipitation

In [ ]:
# Calcualte mean total annual precipitation

# first group by year and sum up values
annual_total_precipitation = daily_precipitation.groupby('time.year').sum(dim='time')
# second calaculate the mean
annual_total_precipitation_mean = annual_total_precipitation.mean(dim='year')
annual_total_precipitation_mean

In [ ]:
# Plot the map of mean total annual precipitation
annual_total_precipitation_mean['tp'].plot.imshow()
plt.title('Mean Total Annual Precipitation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
threshold = 30

#Calaculate the mean annual intensity of precipitation events > threshold
intensity_threshold = daily_precipitation.where(daily_precipitation['tp'] > threshold)
intensity_annual_sum = intensity_threshold.groupby('time.year').sum(dim='time')
intensity_mean = intensity_annual_sum.mean(dim='year')

#Calaculate the mean annual probability of precipitation events > threshold
probability_n = (daily_precipitation['tp'] > threshold)
probability_n_annual_sum = probability_n.groupby('time.year').sum(dim='time')
probability_annual_mean = probability_n_annual_sum / 365 
probability_mean = probability_annual_mean.mean(dim='year')

#probability_mean_xr = xr.Dataset({'intensity':probability_mean.values}, coords={'longitude': probability_mean['longitude'],'latitude': probability_mean['latitude']})
probability_mean_xr = xr.Dataset({'probability':probability_mean})

In [ ]:
# Plot mean annual intensity map of precipitation events > threshold
intensity_mean['tp'].plot.imshow()
plt.title('Mean Annual Intensity of Extreme Precipitation Events')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Plot mean annual probability precipitation events > threshold
probability_mean.plot.imshow()
plt.title('Mean Annual Probability of Extreme Precipitation Events')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Store the mean and median maps as netcdf file in the given folder
path = 'data/results/'
filename = 'ERA5_Tenerife_mean_total_precipitation_'  + str(years[0]) + '_' + str(years[-1]) + '.nc'
annual_total_precipitation_mean.to_netcdf(path + filename)

filename = 'ERA5_Tenerife_intensity_extreme_precipitation_' + str(threshold) + 'mm_'  + str(years[0]) + '_' + str(years[-1]) + '.nc'
intensity_mean.to_netcdf(path + filename)

filename = 'ERA5_Tenerife_probability_extreme_precipitation_' + str(threshold) + 'mm_' + str(years[0]) + '_' + str(years[-1]) + '.nc'
probability_mean_xr.to_netcdf(path + filename)

Now perform the following analysis:
- Calaculate the block maxima with annual frequency for each pixel
- Fit the GEV function to these maxima for each pixel
- Calculate the expected extreme precipitation events for 5, 10 and 20 year return periods
- Create complete maps of the expected extreme precipitation events with QGIS (use the Tenerife Shapefile)

Make use of the [xclim libary](https://xclim.readthedocs.io/en/stable/notebooks/frequency_analysis.html) which is already imported (see script header)

In [ ]:
# Befor the 2D map can be saved we have to get rid
# of the quatile dimension using sel(quantile=pq['quantile'][0], drop=True)

# Save map as new netcdf file in the reult folder e.g. with a file name
# filename = 'ERA5_Tenerife_precipitation_event_return_period_'  + f'T_ {return_period:.0f}' + '.nc'